## Install Gym environments to interact with k8s

In [ ]:
!pip install -e ../../gym_k8s_real

In [ ]:
import gym
import random
import subprocess
import time
import numpy as np
from threading import Lock, Thread
import datetime
import gym_k8s_real

## Packages to build DQN

In [ ]:
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

## Define DQN class - with Q-value

In [ ]:
 class DQNAgent():
    def __init__(self, env, path, episodes, max_env_steps, win_threshold, epsilon_decay,
                 state_size=None, action_size=None, epsilon=1.0, epsilon_min=0.01, 
                 gamma=1, alpha=.01, alpha_decay=.01, batch_size=16, prints=False):
        self.memory = deque(maxlen=100000)
        self.env = env
 
        if state_size is None: 
            self.state_size = self.env.observation_space.n 
        else: 
            self.state_size = state_size
 
        if action_size is None: 
            self.action_size = self.env.action_space.n 
        else: 
            self.action_size = action_size
 
        self.episodes = episodes
        self.env._max_episode_steps = max_env_steps
        self.win_threshold = win_threshold
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.gamma = gamma
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.batch_size = batch_size
        self.path = path                     #location where the model is saved to
        self.prints = prints                 #if true, the agent will print his scores
 
        self.model = self._build_model()
    
    #Build network model
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='tanh'))
        model.add(Dense(48, activation='tanh'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.alpha, decay=self.alpha_decay))
        return model
    
    #Generate one action
    def generate_action(self, state, eps, hAction):
        # epsilon-greey to take best action from action-value function
        if np.random.random() < eps:
            return self.env.action_space.sample()
        # select action with herustic-boosted method
        hVal = self.hValue(state, hAction)
        maxAction = -1
        maxQ = -1
        for action in range(len(self.model.predict(state)[0])):
            aggreVal = self.model.predict(state)[0][action] + hVal if action == hAction else self.model.predict(new_state)[0][action] + 0
            if maxQ < aggreVal:
                maxQ = aggreVal
                maxAction = action
        return maxAction
    
    #Add states into memory
    def remember(self, state, action, reward, next_state, done): 
        self.memory.append((state, action, reward, next_state, done))
    
    #Replay memory to train
    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(
            self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.model.predict(state)
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.model.predict(next_state)[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])

        self.model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    #Define calculate heuristic value; input: action from heuristic policy
    #hAction is returned from gym environment and is encoded 
    def hValue(self, state, hAction):
        #Q-value where action is equal to heuristic action
        heurQ = self.target_model.predict(state)[0][hAction]
        hVal = np.amax(self.target_model.predict(state)[0])
        return hVal - heurQ
        
    #Actual training process
    def learning(self, hAction):
        self.t = (self.t + 1) % self.C
        
        # update every C times and make sure buffer is filled with at least size batch size
        if self.t == 0:
            if len(self.memory) < self.batch_size: 
                return
            
            # init list states to store states 
            # init list of targets values forecast gernated by model Q associated with each state-action
            states, targets_forecast = [], []
            
            # random sample from replay buffer
            samples = random.sample(self.memory, self.batch_size)
            
            for state, action, reward, new_state, done in samples:
                if done:
                    target = reward
                else:
                    Q_new_state =  np.amax(self.target_model.predict(new_state)[0])
                    target = reward + self.gamma *  Q_new_state

                target_forecast = self.model.predict(state)
                target_forecast[0][action] = target
                
                # append to lists for batch processing outside the iteartion
                states.append(state[0])
                targets_forecast.append(target_forecast[0])
            
            # batch learning to train the model Q   
            self.model.fit(np.array(states), np.array(targets_forecast), epochs=1, verbose=0)
            self.train_target()
            
    # soft update to target model Q_hat from model Q
    def train_target(self):
        # target model and model are not updating at the same time
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = self.tau * weights[i] + (1 - self.tau) * target_weights[i]
        # assign new weights to target model
        self.target_model.set_weights(target_weights)
    
    #Save model
    def save_model(self, name='DQN_Model'):
        self.model.save(self.path + name)
    
    

## Info about the kubernetes environment we deploy

In [ ]:
# Timestep duration in minutes
# We wait these many minutes for our actions to be enforced
timestep_duration = 1.5
current_app_names = ["firewall", "encrypt", "decrypt"]
current_cluster_names = ["kubernetes-master-dev", "kubernetes-work-dev"]
memory_req = '128Mi'
cpu_req = '80m'
sla_latency = 2.6
sla_host = 'http://145.100.135.89:6088/'
# latency metric
sla_metric_name = 'latency'
gym_env = 'gym_k8s_real:k8s-env-dqn-v0'
#init Q_table
total_epochs = 100
num_of_services = 1
steps_per_epoch = 15
app_dict = {"firewall": 500, "encrypt": 500, "decrypt": 500}
config_path = "~/EPI_kube_scaling/EPIF-Configurations/"

In [ ]:
env = gym.make(
        gym_env,
        timestep_duration=timestep_duration,
        app_names=current_app_names,
        app_configs = ['s', 'm', 'l'], 
        cluster_names = current_cluster_names, 
        sla_latency = sla_latency,
        sla_host = sla_host, 
        sla_latency_metric_name = sla_metric_name, 
        max_pods = 30, 
        min_pods = 1,
        app_dict = app_dict,
        config_path = config_path,
        changes = [0, 1, 2]
    )

## Create historical states csv file if it doesn't exist

In [ ]:
try:
    open('k8s_historical_states.csv', 'r').close()
    print('File already present.')
except IOError:
    with open('k8s_historical_states.csv', 'w') as f:
        #TO-DO: define states you want to collect
#         f.write()
    print('File not present. Created successfully!')

# Train the agent

## Agent training
This function trains our agent:

In [ ]:
def train_agent():
    
    #make sure env is maked
    env = gym.make(
        gym_env,
        timestep_duration=timestep_duration,
        app_name=current_app_name,
        sla_throughput=sla_latency,
        prometheus_host=sla_host,
        prometheus_throughput_metric_name=current_throughput_metric_name
    )
    
    #define the DQN agent
    agent = DQNAgent(env, "", 100, 10, 0.1, 10,
                 state_size=None, action_size=None, epsilon=1.0, epsilon_min=0.01, 
                 gamma=1, alpha=.01, alpha_decay=.01, batch_size=16, prints=False)
    
    for epoch in range(0, total_epochs):
        state, _ = env.reset()
        #TODO: get hAction from heuristic method
        hAction = 0
        
        done = False
        

        for step in range(steps_per_epoch):
            current_timestep = epoch * steps_per_epoch + step
        
            
            if done:
                break
            else: 
                action = agent.generate_action(state, epsilon, hAction)
                        
                real_ob, reward, done, next_state = env.step(action)
                    
                #Update Q-value
                agent.remember(state, action, reward, next_state, done)
                
                #Try one round of training
                agent.learning()
                
                #To-do: Update H-value
            
                #Update state
                state = next_state
                
        agent.save_model()
        print('One epoch of training finished.\n')

In [ ]:
train_agent()